In [10]:
# import requests
import json

In [11]:
# docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
# docs_response = requests.get(docs_url)
# documents_raw = docs_response.json()


doc_path = '../week1/documents-llm.json'
with open(doc_path, 'rb') as doc:
    documents_raw = json.load(doc)

len(documents_raw)

1

In [12]:
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
# `docker pull qdrant/qdrant

# docker run -p 6333:6333 -p 6334:6334 \
#    -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
#    qdrant/qdrant`

In [16]:
from qdrant_client import QdrantClient

q_client = QdrantClient("http://localhost:6333")
q_client.get_collections()

CollectionsResponse(collections=[])

# Sparse vector search with BM25
- This uses TF-IDF vectorization which is handled by Qdrant

In [17]:
from qdrant_client import models

# Create the collection with specified sparse vector parameters
q_client.create_collection(
    collection_name="zoomcamp-sparse",
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    }
)

True

In [21]:
import uuid

# Send the points to the collection
q_client.upsert(
    collection_name="zoomcamp-sparse",
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "bm25": models.Document(
                    text=doc["text"], 
                    model="Qdrant/bm25",
                ),
            },
            payload={
                "text": doc["text"],
                "section": doc["section"],
                "course": course["course"],
            }
        )
        for course in documents_raw
        for doc in course["documents"]
    ]
)

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]c:\Users\USER\Desktop\Isah\llm-zoomcamp2\llm-coursework\llm-env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\AppData\Local\Temp\fastembed_cache\models--Qdrant--bm25. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 18 files:  11%|█

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [46]:
def search(query: str, limit: int = 1) -> list[models.ScoredPoint]:
    results = q_client.query_points(
        collection_name="zoomcamp-sparse",
        query=models.Document(
            text=query,
            model="Qdrant/bm25",
        ),
        using="bm25",
        limit=limit,
        with_payload=True,
    )

    return results.points

In [47]:
results = search("Qdrant")
results

[ScoredPoint(id='91151618-79c7-4536-8e84-e6bd47fa7dfd', version=0, score=7.0953813, payload={'text': 'No, the capstone is a solo project.', 'section': 'Capstone Project', 'course': 'llm-zoomcamp'}, vector=None, shard_key=None, order_value=None)]

In [48]:
results = search("pandas")
print(results[0].payload["text"])

No, the capstone is a solo project.


In [49]:
results[0].score

7.0953813

In [50]:
import random
# import json

random.seed(202506)

course = random.choice(documents_raw)
course_piece = random.choice(course["documents"])
print(json.dumps(course_piece, indent=2))

{
  "text": "Use the service name and port provided in the docker-compose.yaml file for the elasticsearch, e.g <http://><docker-compose-service-name>:<port> <http://elasticsearch:9200>",
  "section": "Module 5: X",
  "question": "I have reached the orchestration pipeline's export and I\u2019m facing a connection error at the stage of exporting to the vector database. Can someone help with the connection string?",
  "course": "llm-zoomcamp"
}


In [52]:
results = search(course_piece["question"])
print(results[0].payload["text"])

No, the capstone is a solo project.


In [53]:
# Create the collection with both vector types
q_client.create_collection(
    collection_name="zoomcamp-sparse-and-dense",
    vectors_config={
        # Named dense vector for jinaai/jina-embeddings-v2-small-en
        "jina-small": models.VectorParams(
            size=512,
            distance=models.Distance.COSINE,
        ),
    },
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    }
)

True

In [55]:
q_client.upsert(
    collection_name="zoomcamp-sparse-and-dense",
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "jina-small": models.Document(
                    text=doc["text"],
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                "bm25": models.Document(
                    text=doc["text"], 
                    model="Qdrant/bm25",
                ),
            },
            payload={
                "text": doc["text"],
                "section": doc["section"],
                "course": course["course"],
            }
        )
        for course in documents_raw
        for doc in course["documents"]
    ]
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\USER\Desktop\Isah\llm-zoomcamp2\llm-coursework\llm-env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\AppData\Local\Temp\fastembed_cache\models--xenova--jina-embeddings-v2-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activat

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [56]:
def multi_stage_search(query: str, limit: int = 1) -> list[models.ScoredPoint]:
    results = q_client.query_points(
        collection_name="zoomcamp-sparse-and-dense",
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                using="jina-small",
                # Prefetch ten times more results, then
                # expected to return, so we can really rerank
                limit=(10 * limit),
            ),
        ],
        query=models.Document(
            text=query,
            model="Qdrant/bm25", 
        ),
        using="bm25",
        limit=limit,
        with_payload=True,
    )

    return results.points

In [57]:
print(json.dumps(course_piece, indent=2))

{
  "text": "Use the service name and port provided in the docker-compose.yaml file for the elasticsearch, e.g <http://><docker-compose-service-name>:<port> <http://elasticsearch:9200>",
  "section": "Module 5: X",
  "question": "I have reached the orchestration pipeline's export and I\u2019m facing a connection error at the stage of exporting to the vector database. Can someone help with the connection string?",
  "course": "llm-zoomcamp"
}


In [58]:
results = multi_stage_search(course_piece["question"])
print(results[0].payload["text"])

Use the following line instead in mounting the current volume to docker for Q4:
`-v "/${PWD}/ollama_files:/root/.ollama"`


In [59]:
def rrf_search(query: str, limit: int = 1) -> list[models.ScoredPoint]:
    results = q_client.query_points(
        collection_name="zoomcamp-sparse-and-dense",
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                using="jina-small",
                limit=(5 * limit),
            ),
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="Qdrant/bm25",
                ),
                using="bm25",
                limit=(5 * limit),
            ),
        ],
        # Fusion query enables fusion on the prefetched results
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        with_payload=True,
    )

    return results.points

In [60]:
results = rrf_search(course_piece["question"])
print(json.dumps(course_piece, indent=2))
print(results[0].payload["text"])

{
  "text": "Use the service name and port provided in the docker-compose.yaml file for the elasticsearch, e.g <http://><docker-compose-service-name>:<port> <http://elasticsearch:9200>",
  "section": "Module 5: X",
  "question": "I have reached the orchestration pipeline's export and I\u2019m facing a connection error at the stage of exporting to the vector database. Can someone help with the connection string?",
  "course": "llm-zoomcamp"
}
By default, in the dataframe visualization, Pandas truncate the text content in a column to 50 characters. In order to view the entire explanation given by the judge llm for a NON RELEVANT answer, as in figure:
The instruction to show the results must be preceded by:
pd.set_option('display.max_colwidth', None)
Here are the specs for the display_max_colwidth option, as describide in the official docs:
display.max_colwidth : int or None
The maximum width in characters of a column in the repr of
a pandas data structure. When the column overflows, a ".